In [73]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns in DataFrame display
pd.set_option('display.width', None)        # Don't truncate wide rows


In [74]:
import pandas as pd
import numpy as np
import asyncio
from KNN_faiss import GeoKNNSearch
from property_scraper import PropertyScraper
import re  # Import the scraper class
from property_analyzer import PropertyAnalyzer

In [75]:
def load_data():
    """Loads property dataset and splits into recent (2025) and historical records."""
    data_enrich_train = pd.read_excel("Data/final_merged_output.xlsx")

    return data_enrich_train

In [76]:
def initialize_knn(data_enrich_train):
    """Initializes the KNN model with historical property data."""
    return GeoKNNSearch(
        data=data_enrich_train[['Property_Index', 'position.lat', 'position.long']],
        lat_col='position.lat', lon_col='position.long', id_col='Property_Index',
        use_exact_distance=True
    )

In [77]:
df=load_data()
df = df.iloc[1:].reset_index(drop=True)

In [78]:
columns_to_keep = [
    "Matched_PID",
    "Input_PAON",
    "Input_SAON",
    "Input_Street",
    "Input_Postcode",
    "Property_Index",
    "property.type",
    "property.tenure",
    "property.age",
    "position.lat",
    "position.long",
    "property.highestPricePaid.value",
    "property.highestPricePaid.amount",
    "property.lowestPricePaid.value",
    "property.lowestPricePaid.amount",
    "property.estimatedCurrentValue.value",
    "property.estimatedCurrentValue.amount",
    "property.estimatedCurrentValue.floorArea",
    "property.estimatedCurrentValue.pricePerSqm",
    "property.estimatedCurrentValue.confidence"
]
df = df[columns_to_keep]


In [79]:
sample = df.iloc[0]
latitude=[sample["position.lat"]]
longitude=[sample["position.long"]]
latitude_np = np.float64(latitude[0])
longitude_np=np.float64(longitude[0])

In [94]:
subject_property_coordinates = (latitude_np, longitude_np)
knn = initialize_knn(df)
property_indices, distances = knn.knearest(subject_property_coordinates, 500, return_distances=True)
# Convert to a dictionary to get unique indices and their minimum distances
unique_distances = {}
for index, distance in zip(property_indices, distances):
    if index in unique_distances:
        if unique_distances[index] != distance:
            print(f"Conflict for index {index}: existing distance {unique_distances[index]}, new distance {distance}")
    else:
        unique_distances[index] = distance

potential_comparables = np.array(list(unique_distances.items()))

In [95]:
new_df = pd.DataFrame(potential_comparables, columns=['Property_Index', 'Physical Distance'])

# Convert 'Physical Distance' to float
new_df['Physical Distance'] = new_df['Physical Distance'].astype(np.float64)

# Now merge with df on Property_Index
pc_df = df.merge(new_df, on='Property_Index', how='inner')

In [96]:
new_df = pd.read_excel("Data/premium_property_enrich_individual_properties.xlsx")

# Step 2: Ensure 'Property_Index' exists in both DataFrames
if 'Property_Index' not in new_df.columns:
    raise ValueError("The new dataframe does not have a 'Property_Index' column.")

# Step 3: Group by 'Property_Index' and sum the 'Price' column
price_df = new_df.groupby('Property_Index', as_index=False)['Price'].sum()

# Step 4: Merge with pc_df
pc_with_price = pc_df.merge(price_df, on='Property_Index', how='left')

# Step 5: Identify and report unmatched rows
unmatched = pc_with_price[pc_with_price['Price'].isna()]
if not unmatched.empty:
    print("Unmatched Property_Index rows:")
    print(unmatched['Property_Index'].tolist())
    print(f"Total unmatched rows: {len(unmatched)}")

# Drop unmatched rows where 'Price' is NaN
pc_with_price = pc_with_price.dropna(subset=['Price']).reset_index(drop=True)


Unmatched Property_Index rows:
['SW15 1AL EGLISTON LAWNS 13 3', 'SW6 3LF CAWTHORN APARTMENTS 86 42', 'SW18 1LL CHARTFIELD HOUSE 27', 'SW18 1UJ COOPERS LOFTS 5 APARTMENT 9', 'SW18 1UJ COOPERS LOFTS 5 APARTMENT 9', 'SW6 6NZ ELM LODGE 75 FLAT 9', 'SW15 2JJ CASTLE COURT 1 FLAT 49']
Total unmatched rows: 7


In [97]:
# Step 1: Sort by physical distance
nearest_5 = pc_with_price.sort_values(by='Physical Distance').head(10).copy()

# Step 2: Avoid division by zero (if distance = 0, add a tiny number)
nearest_5['Weight'] = 1 / (nearest_5['Physical Distance'] + 1e-6)

# Step 3: Calculate weighted average price
weighted_mean_price = (nearest_5['Price'] * nearest_5['Weight']).sum() / nearest_5['Weight'].sum()

# Step 4: Define 50% price bounds
lower_bound = weighted_mean_price * 0.5
upper_bound = weighted_mean_price * 1.5

# Step 5: Filter original dataframe based on those bounds
final_potential_comparables = pc_with_price[
    (pc_with_price['Price'] >= lower_bound) &
    (pc_with_price['Price'] <= upper_bound)
]

# Optional: print weighted mean and bounds
print(f"Weighted Mean Price: {weighted_mean_price:.2f}")
print(f"Allowed Price Range: {lower_bound:.2f} - {upper_bound:.2f}")


Weighted Mean Price: 2775726.61
Allowed Price Range: 1387863.31 - 4163589.92


In [98]:
# Step 6: Find additional rows matching Street, excluding same Property_Index and already-included ones
additional_rows = pc_with_price[
    (pc_with_price['Input_Street'] == sample['Input_Street']) &
    (pc_with_price['Property_Index'] != sample['Property_Index']) &
    (~pc_with_price['Property_Index'].isin(final_potential_comparables['Property_Index']))
]

# Step 7: Add those rows to final_potential_comparables
final_potential_comparables = pd.concat([final_potential_comparables, additional_rows], ignore_index=True)


In [100]:
import pandas as pd
import numpy as np

# Define columns to check
columns_to_check = [
    'property.type', 'property.tenure', 'property.age', 'position.lat', 'position.long',
    'property.highestPricePaid.value', 'property.highestPricePaid.amount',
    'property.lowestPricePaid.value', 'property.lowestPricePaid.amount',
    'property.estimatedCurrentValue.value', 'property.estimatedCurrentValue.amount',
    'property.estimatedCurrentValue.floorArea', 'property.estimatedCurrentValue.pricePerSqm',
    'property.estimatedCurrentValue.confidence', 'Physical Distance', 'Price'
]

# Step 1: Replace blank strings with NaN
df_cleaned = final_potential_comparables.copy()
df_cleaned[columns_to_check] = df_cleaned[columns_to_check].replace(r'^\s*$', np.nan, regex=True)

# Step 2: Count rows before dropping
initial_rows = df_cleaned.shape[0]

# Step 3: Drop rows with any NaN in the selected columns
df_cleaned = df_cleaned.dropna(subset=columns_to_check).reset_index(drop=True)

# Step 4: Count rows after dropping
final_rows = df_cleaned.shape[0]
dropped_rows = initial_rows - final_rows

print(f"Removed {dropped_rows} rows due to missing or invalid values in critical columns.")

# Optional: Assign back to your original variable
final_potential_comparables = df_cleaned


Removed 9 rows due to missing or invalid values in critical columns.


C:\Users\ArjunShankar\AppData\Local\Temp\ipykernel_21488\2230651290.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned[columns_to_check] = df_cleaned[columns_to_check].replace(r'^\s*$', np.nan, regex=True)


In [102]:
final_potential_comparables.columns

Index(['Matched_PID', 'Input_PAON', 'Input_SAON', 'Input_Street',
       'Input_Postcode', 'Property_Index', 'property.type', 'property.tenure',
       'property.age', 'position.lat', 'position.long',
       'property.highestPricePaid.value', 'property.highestPricePaid.amount',
       'property.lowestPricePaid.value', 'property.lowestPricePaid.amount',
       'property.estimatedCurrentValue.value',
       'property.estimatedCurrentValue.amount',
       'property.estimatedCurrentValue.floorArea',
       'property.estimatedCurrentValue.pricePerSqm',
       'property.estimatedCurrentValue.confidence', 'Physical Distance',
       'Price'],
      dtype='object')

In [103]:
final_potential_comparables

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,property.highestPricePaid.value,property.highestPricePaid.amount,property.lowestPricePaid.value,property.lowestPricePaid.amount,property.estimatedCurrentValue.value,property.estimatedCurrentValue.amount,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,property.estimatedCurrentValue.confidence,Physical Distance,Price
0,c95444463147530ca2ace31548b14658,57,NaN,st johns avenue,sw15 6al,SW15 6AL 57,Detached,Freehold,2007 onwards,51.459540,-0.221640,2765000.0,"£2,765,000",1700000.0,"£1,700,000",3104865.0,"£3,104,865",287.0,10818.344948,High,0.000136,2765000.0
1,8230b33a05109321e63e819efeae99c3,37,NaN,melrose road,sw18 1lx,SW18 1LX 37,Detached,Freehold,1900-1929,51.451812,-0.205756,2750000.0,"£2,750,000",1850000.0,"£1,850,000",3144625.0,"£3,144,625",258.0,12188.468992,High,1.396289,2750000.0
2,0df927938fcace3143324a9d12968e77,29,NaN,hotham road,sw15 1ql,SW15 1QL 29,Semi Detached,Freehold,before 1900,51.465129,-0.223932,2015000.0,"£2,015,000",500000.0,"£500,000",2339562.0,"£2,339,562",245.0,9549.232653,High,0.641505,2015000.0
3,57974246df65cb387ccd9dacbe0f13ef,4,NaN,bellevue road,sw13 0bj,SW13 0BJ 4,Mid Terrace,Freehold,1900-1929,51.471997,-0.240512,2150000.0,"£2,150,000",2150000.0,"£2,150,000",2502930.0,"£2,502,930",208.0,12033.317308,High,1.904578,2150000.0
4,7b3b49d08b2e3a5ab7bfb45cd1877981,31,NaN,west hill road,sw18 1ll,SW18 1LL 31,Detached,Freehold,INVALID!,51.453506,-0.205442,2400000.0,"£2,400,000",1450000.0,"£1,450,000",2695000.0,"£2,695,000",234.0,11517.094017,High,1.307560,2400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,f59467c254f31024fe380d4c75119b5a,14,NaN,bellevue road,sw13 0bj,SW13 0BJ 14,Mid Terrace,Freehold,1900-1929,51.472233,-0.240662,2555000.0,"£2,555,000",740000.0,"£740,000",2564496.0,"£2,564,496",199.0,12886.914573,High,1.930833,2555000.0
413,eb2b010a895a68036fb9ee1c55128d5f,8,NaN,wedgewood mews,sw6 3eq,SW6 3EQ 8,Flat/Maisonette,Freehold,2019,51.469904,-0.211009,2200000.0,"£2,200,000",1875000.0,"£1,875,000",2176208.0,"£2,176,208",155.0,14040.051613,High,1.367452,2200000.0
414,90808fa150f5e552d63f4852f85ffee0,23,NaN,cristowe road,sw6 3qf,SW6 3QF 23,Mid Terrace,Freehold,1900-1929,51.471135,-0.204404,2383405.0,"£2,383,405",895000.0,"£895,000",2429839.0,"£2,429,839",228.0,10657.188596,High,1.757188,2383405.0
415,f4c14c9dfedc2390645e65fae192f79b,51,NaN,disraeli road,sw15 2dr,SW15 2DR 51,Mid Terrace,Freehold,before 1900,51.461268,-0.213702,2750000.0,"£2,750,000",675000.0,"£675,000",2725250.0,"£2,725,250",276.0,9874.094203,High,0.582596,2750000.0


In [108]:
import pandas as pd
import numpy as np

# Columns
categorical_cols = ['property.type', 'property.tenure', 'property.age']
amount_cols = [
    'property.highestPricePaid.value', 'property.highestPricePaid.amount',
    'property.lowestPricePaid.value', 'property.lowestPricePaid.amount',
    'property.estimatedCurrentValue.value', 'property.estimatedCurrentValue.amount',
    'property.estimatedCurrentValue.floorArea', 'property.estimatedCurrentValue.pricePerSqm',
    'Physical Distance'
]

# Make a copy
df = final_potential_comparables.copy()

# === Step 1: Clean and convert amount columns ===
def clean_currency(val):
    if isinstance(val, str):
        val = val.replace("£", "").replace(",", "").strip()
    try:
        return float(val)
    except:
        return np.nan

for col in amount_cols:
    df[col] = df[col].apply(clean_currency)

# === Step 2: Encode categorical columns ===
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df[categorical_cols] = encoder.fit_transform(df[categorical_cols])

# === Step 3: Apply same transformation to sample (Series) ===
def preprocess_sample(sample, encoder, categorical_cols, amount_cols):
    sample = sample.copy()

    # Clean amounts
    for col in amount_cols:
        val = sample[col]
        if isinstance(val, str):
            val = val.replace("£", "").replace(",", "").strip()
        try:
            sample[col] = float(val)
        except:
            sample[col] = np.nan

    # Encode categoricals
    cat_values = sample[categorical_cols].values.reshape(1, -1)
    sample[categorical_cols] = encoder.transform(cat_values)[0]

    return sample

# Apply to sample
sample = preprocess_sample(sample, encoder, categorical_cols, amount_cols)


c:\Users\ArjunShankar\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


In [107]:
sample['Physical Distance']=0.0

In [109]:
df

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,property.highestPricePaid.value,property.highestPricePaid.amount,property.lowestPricePaid.value,property.lowestPricePaid.amount,property.estimatedCurrentValue.value,property.estimatedCurrentValue.amount,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,property.estimatedCurrentValue.confidence,Physical Distance,Price
0,c95444463147530ca2ace31548b14658,57,NaN,st johns avenue,sw15 6al,SW15 6AL 57,0.0,0.0,10.0,51.459540,-0.221640,2765000.0,2765000.0,1700000.0,1700000.0,3104865.0,3104865.0,287.0,10818.344948,High,0.000136,2765000.0
1,8230b33a05109321e63e819efeae99c3,37,NaN,melrose road,sw18 1lx,SW18 1LX 37,0.0,0.0,0.0,51.451812,-0.205756,2750000.0,2750000.0,1850000.0,1850000.0,3144625.0,3144625.0,258.0,12188.468992,High,1.396289,2750000.0
2,0df927938fcace3143324a9d12968e77,29,NaN,hotham road,sw15 1ql,SW15 1QL 29,6.0,0.0,17.0,51.465129,-0.223932,2015000.0,2015000.0,500000.0,500000.0,2339562.0,2339562.0,245.0,9549.232653,High,0.641505,2015000.0
3,57974246df65cb387ccd9dacbe0f13ef,4,NaN,bellevue road,sw13 0bj,SW13 0BJ 4,4.0,0.0,0.0,51.471997,-0.240512,2150000.0,2150000.0,2150000.0,2150000.0,2502930.0,2502930.0,208.0,12033.317308,High,1.904578,2150000.0
4,7b3b49d08b2e3a5ab7bfb45cd1877981,31,NaN,west hill road,sw18 1ll,SW18 1LL 31,0.0,0.0,16.0,51.453506,-0.205442,2400000.0,2400000.0,1450000.0,1450000.0,2695000.0,2695000.0,234.0,11517.094017,High,1.307560,2400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,f59467c254f31024fe380d4c75119b5a,14,NaN,bellevue road,sw13 0bj,SW13 0BJ 14,4.0,0.0,0.0,51.472233,-0.240662,2555000.0,2555000.0,740000.0,740000.0,2564496.0,2564496.0,199.0,12886.914573,High,1.930833,2555000.0
413,eb2b010a895a68036fb9ee1c55128d5f,8,NaN,wedgewood mews,sw6 3eq,SW6 3EQ 8,3.0,0.0,13.0,51.469904,-0.211009,2200000.0,2200000.0,1875000.0,1875000.0,2176208.0,2176208.0,155.0,14040.051613,High,1.367452,2200000.0
414,90808fa150f5e552d63f4852f85ffee0,23,NaN,cristowe road,sw6 3qf,SW6 3QF 23,4.0,0.0,0.0,51.471135,-0.204404,2383405.0,2383405.0,895000.0,895000.0,2429839.0,2429839.0,228.0,10657.188596,High,1.757188,2383405.0
415,f4c14c9dfedc2390645e65fae192f79b,51,NaN,disraeli road,sw15 2dr,SW15 2DR 51,4.0,0.0,17.0,51.461268,-0.213702,2750000.0,2750000.0,675000.0,675000.0,2725250.0,2725250.0,276.0,9874.094203,High,0.582596,2750000.0
